In [12]:
import pandas as pd
import os

In [13]:
path_dataset = os.path.join(os.path.dirname(os.getcwd()), 'Dataset')

In [14]:
# remove "title" and "content" from the content
# remove "The user" from the beginning of the summary

df_issues = pd.read_json(os.path.join(path_dataset, 'issues_original.json'))
df_questions = pd.read_json(os.path.join(path_dataset, 'questions_original.json'))

df_issues['Issue_original_content'] = df_issues['Issue_original_content'].apply(
    lambda x: x.replace('Title: ', '').replace('Content: ', ''))
df_issues['Issue_original_content_gpt_summary'] = df_issues['Issue_original_content_gpt_summary'].apply(
    lambda x: x.removeprefix('The user '))
df_issues['Issue_preprocessed_content'] = df_issues['Issue_preprocessed_content'].apply(
    lambda x: x.replace('Title: ', '').replace('Content: ', ''))

df_questions['Question_original_content'] = df_questions['Question_original_content'].apply(
    lambda x: x.replace('Title: ', '').replace('Content: ', ''))
df_questions['Question_original_content_gpt_summary'] = df_questions['Question_original_content_gpt_summary'].apply(
    lambda x: x.removeprefix('The user '))
df_questions['Question_preprocessed_content'] = df_questions['Question_preprocessed_content'].apply(
    lambda x: x.replace('Title: ', '').replace('Content: ', ''))

df_issues['Challenge_original_content'] = df_issues['Issue_original_content']
df_issues['Challenge_original_content_gpt_summary'] = df_issues['Issue_original_content_gpt_summary']
df_issues['Challenge_preprocessed_content'] = df_issues['Issue_preprocessed_content']

df_questions['Challenge_original_content'] = df_questions['Question_original_content']
df_questions['Challenge_original_content_gpt_summary'] = df_questions['Question_original_content_gpt_summary']
df_questions['Challenge_preprocessed_content'] = df_questions['Question_preprocessed_content']

del df_issues['Issue_original_content']
del df_issues['Issue_original_content_gpt_summary']
del df_issues['Issue_preprocessed_content']

del df_questions['Question_original_content']
del df_questions['Question_original_content_gpt_summary']
del df_questions['Question_preprocessed_content']

df_questions['Solution_original_content'] = df_questions['Answer_original_content']
df_questions['Solution_original_content_gpt_summary'] = df_questions['Answer_original_content_gpt_summary']
df_questions['Solution_preprocessed_content'] = df_questions['Answer_preprocessed_content']

del df_questions['Answer_original_content']
del df_questions['Answer_original_content_gpt_summary']
del df_questions['Answer_preprocessed_content']

df_all = pd.concat([df_issues, df_questions], ignore_index=True)
df_all.to_json(os.path.join(path_dataset, 'all_original.json'), indent=4, orient='records')


In [9]:
# visualize the best topic model

from sklearn.feature_extraction.text import TfidfVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
from sentence_transformers import SentenceTransformer
from bertopic.representation import KeyBERTInspired
from bertopic import BERTopic
from hdbscan import HDBSCAN
from umap import UMAP

# Step 1 - Extract embeddings
embedding_model = SentenceTransformer("all-mpnet-base-v2")

# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=10, n_components=2,
                  metric='manhattan', low_memory=False)

# Step 3 - Cluster reduced embeddings
hdbscan_model = HDBSCAN(min_cluster_size=50, max_cluster_size=1500)

# Step 4 - Tokenize topics
vectorizer_model = TfidfVectorizer(stop_words="english", ngram_range=(1, 4))

# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

# Step 6 - (Optional) Fine-tune topic representation
representation_model = KeyBERTInspired()

# All steps together
topic_model = BERTopic(
    embedding_model=embedding_model,            # Step 1 - Extract embeddings
    umap_model=umap_model,                      # Step 2 - Reduce dimensionality
    hdbscan_model=hdbscan_model,                # Step 3 - Cluster reduced embeddings
    vectorizer_model=vectorizer_model,          # Step 4 - Tokenize topics
    ctfidf_model=ctfidf_model,                  # Step 5 - Extract topic words
    # Step 6 - (Optional) Fine-tune topic represenations
    representation_model=representation_model,
    # verbose=True                              # Step 7 - Track model stages
)

df_challenges = pd.read_json(os.path.join(path_dataset, 'all_original.json'))
docs = df_challenges['Challenge_original_content_gpt_summary'].tolist()

topic_model = topic_model.fit(docs)
# topic_model.save(os.path.join(path_dataset, 'Topic model'))

# fig = topic_model.visualize_topics()
# fig.write_html(os.path.join(path_dataset, 'Topic visualization.html'))

# fig = topic_model.visualize_barchart()
# fig.write_html(os.path.join(path_dataset, 'Term visualization.html'))

# fig = topic_model.visualize_heatmap()
# fig.write_html(os.path.join(path_dataset, 'Topic similarity visualization.html'))

# fig = topic_model.visualize_term_rank()
# fig.write_html(os.path.join(path_dataset, 'Term score decline visualization.html'))

# hierarchical_topics = topic_model.hierarchical_topics(docs)
# fig = topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)
# fig.write_html(os.path.join(path_dataset, 'Hierarchical clustering visualization.html'))

# embeddings = embedding_model.encode(docs, show_progress_bar=False)
# fig = topic_model.visualize_documents(docs, embeddings=embeddings)
# fig.write_html(os.path.join(path_dataset, 'Document visualization.html'))

info_df = topic_model.get_topic_info()
info_df


,Topic,Count,Name
0,-1,5050,-1_azure_attempting deploy_kubeflow_mlflow
1,0,368,0_logging trained model mlflow_using mlflow_lo...
2,1,322,1_git repository_git lfs_git repo_data version...
3,2,220,2_jupyter notebook instance_run jupyter notebo...
4,3,207,3_challenges accessing s3_s3 bucket encounteri...
5,4,204,4_gpu utilization_average gpu utilization_aver...
6,5,191,5_features improvements bug fixes_major featur...
7,6,174,6_azure data lake gen2_azure data lake storage...
8,7,168,7_deploy trained model endpoint_multi model en...
9,8,162,8____


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.coherencemodel import CoherenceModel
from bertopic.vectorizers import ClassTfidfTransformer
from sentence_transformers import SentenceTransformer
from bertopic.representation import KeyBERTInspired
from bertopic import BERTopic
from hdbscan import HDBSCAN
from umap import UMAP

import gensim.corpora as corpora
import pandas as pd
import wandb
import os

os.environ["WANDB_API_KEY"] = '9963fa73f81aa361bdbaf545857e1230fc74094c'
os.environ["WANDB_AGENT_MAX_INITIAL_FAILURES"]= "150"
os.environ["TOKENIZERS_PARALLELISM"] = "true"

wandb_project = 'asset-management-project'
wandb.login()

path_dataset = os.path.join(os.path.dirname(os.getcwd()), 'Dataset')
df_all = pd.read_json(os.path.join(path_dataset, 'all_original.json'))
docs = df_all['Challenge_original_content_gpt_summary'].tolist()

# set general sweep configuration
sweep_configuration = {
    "name": "experiment-2",
    "metric": {
        'name': 'CoherenceCV',
        'goal': 'maximize'
    },
    "method": "grid",
    "parameters": {
        'n_neighbors': {
            'values': list(range(10, 110, 10))
        },
        'n_components': {
            'values': list(range(2, 12, 2))
        },
        'ngram_range': {
            'values': list(range(3, 6))
        },
    }
}

# set default sweep configuration
config_defaults = {
    'model_name': 'all-mpnet-base-v2',
    'metric_distane': 'manhattan',
    'low_memory': True,
    'max_cluster_size': 1500,
    'min_cluster_size': 50,
    'stop_words': 'english',
    'reduce_frequent_words': True
}


def train():
    # Initialize a new wandb run
    with wandb.init() as run:
        # update any values not set by sweep
        run.config.setdefaults(config_defaults)

        # Step 1 - Extract embeddings
        embedding_model = SentenceTransformer(run.config.model_name)

        # Step 2 - Reduce dimensionality
        umap_model = UMAP(n_neighbors=wandb.config.n_neighbors, n_components=wandb.config.n_components,
                          metric=run.config.metric_distane, low_memory=run.config.low_memory)

        # Step 3 - Cluster reduced embeddings
        hdbscan_model = HDBSCAN()

        # Step 4 - Tokenize topics
        vectorizer_model = TfidfVectorizer(
            stop_words=run.config.stop_words, ngram_range=(1, wandb.config.ngram_range))

        # Step 5 - Create topic representation
        ctfidf_model = ClassTfidfTransformer(
            reduce_frequent_words=run.config.reduce_frequent_words)

        # Step 6 - Fine-tune topic representation
        representation_model = KeyBERTInspired()

        # All steps together
        topic_model = BERTopic(
            embedding_model=embedding_model,
            umap_model=umap_model,
            hdbscan_model=hdbscan_model,
            vectorizer_model=vectorizer_model,
            ctfidf_model=ctfidf_model,
            representation_model=representation_model,
            # Step 7 - Track model stages
            # verbose=True
        )

        topics, _ = topic_model.fit_transform(docs)

        # Preprocess documents
        documents = pd.DataFrame(
            {"Document": docs,
             "ID": range(len(docs)),
             "Topic": topics}
        )
        documents_per_topic = documents.groupby(
            ['Topic'], as_index=False).agg({'Document': ' '.join})
        cleaned_docs = topic_model._preprocess_text(
            documents_per_topic.Document.values)

        # Extract vectorizer and analyzer from fit model
        analyzer = vectorizer_model.build_analyzer()
        # Extract features for topic coherence evaluation
        tokens = [analyzer(doc) for doc in cleaned_docs]
        dictionary = corpora.Dictionary(tokens)
        corpus = [dictionary.doc2bow(token) for token in tokens]
        topic_words = [[words for words, _ in topic_model.get_topic(topic)]
                       for topic in range(len(set(topics))-1)]

        coherence_cv = CoherenceModel(
            topics=topic_words,
            texts=tokens,
            corpus=corpus,
            dictionary=dictionary,
            coherence='c_v'
        )

        coherence_umass = CoherenceModel(
            topics=topic_words,
            texts=tokens,
            corpus=corpus,
            dictionary=dictionary,
            coherence='u_mass'
        )

        coherence_cuci = CoherenceModel(
            topics=topic_words,
            texts=tokens,
            corpus=corpus,
            dictionary=dictionary,
            coherence='c_uci'
        )

        coherence_cnpmi = CoherenceModel(
            topics=topic_words,
            texts=tokens,
            corpus=corpus,
            dictionary=dictionary,
            coherence='c_npmi'
        )

        wandb.log({'CoherenceCV': coherence_cv.get_coherence()})
        wandb.log({'CoherenceUMASS': coherence_umass.get_coherence()})
        wandb.log({'CoherenceUCI': coherence_cuci.get_coherence()})
        wandb.log({'CoherenceNPMI': coherence_cnpmi.get_coherence()})


sweep_id = wandb.sweep(sweep_configuration, project=wandb_project)
# Create sweep with ID: j7pnz7gn
wandb.agent(sweep_id=sweep_id, function=train)
